# CNN Results and Evaluation

In [1]:
# load the necessary packages
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

import itertools

import os
import numpy as np
import matplotlib.pyplot as plt

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


KeyboardInterrupt: 

### Load Data and Model

In [ ]:
batch_size = 100
IMG_HEIGHT = 180
IMG_WIDTH = 180
test_dir = 'data/resized/test'
dropout_rate = 0.15

In [ ]:
test_image_generator = ImageDataGenerator(rescale=1./255)
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=test_dir,
                                                          shuffle=False,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH))

In [ ]:
test_data_gen.samples

In [ ]:
# load up saved model
model = load_model('saved_models/CNN_MODEL_V1.h5')

In [ ]:
# Visualize training history
# list all data in history
print(model.history.keys())
# summarize history for accuracy
plt.plot(model.history['accuracy'])
plt.plot(model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(model.history['loss'])
plt.plot(model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# make predictions on test set
probabilities = model.predict_generator(test_data_gen,
                                        steps=25250//batch_size + 1)

In [ ]:
y_pred = np.argmax(probabilities, axis=1)
print("Total number of test pictures: {}".format(len(y_pred)))

In [ ]:
# set true labels
y_test = test_data_gen.classes
y_test

In [ ]:
# initialize class names
class_names = []
for key in test_data_gen.class_indices.keys():
    class_names.append(key)

### Get stats on prediction per class

In [ ]:
print('Classification Report')
print(classification_report(y_test, y_pred, target_names=class_names))

Taking a look at the results we can start to understand how the model is viewing the images. 

The top 3 classes to be identified correctly are edamame, spaghetti carbonara, and seaweed salad. These three foods definitely have more unique looks compared to other types of food. 

The top 3 classes to be misidentified are steak, grilled salmon, and breakfast burritos. More examination will be required to understand why these classes are being misidentified so much.

### Plot confusion Matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plt.figure()
fig = plt.gcf()
fig.set_size_inches(32, 32)
plot_confusion_matrix(confusion_matrix(y_test, y_pred), classes=class_names,
                      title='Confusion matrix, without normalization',
                      cmap=plt.cm.GnBu)
plt.show()

This confusion martrix plots the true identifications on the diagonal and the wrong classificationson every other spot. The Columns represent the predicted class and the rows represent the actual class.

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
auc = multiclass_roc_auc_score(y_test, y_pred)
print("Multi-Class AUC Score is: {}".format(auc))

There is a multi-class AUC score of .69 which means the predictions are accurate. However as seen above this is not always the case.

In [ ]:
def show_images_prediction(food_class,y_test, page=0):
    page_size = 20
    nrows = 4
    ncols = 5
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 12))
    fig.set_size_inches(12, 8)
    fig.tight_layout()
    start_i = (class_names.index(food_class) * 250) + (page * 20)
    for i, ax in enumerate(axes.flat):
        im = ax.imshow(plt.imread(test_data_gen.filepaths[i+start_i]))
        ax.set_axis_off()
        ax.title.set_visible(False)
        ax.xaxis.set_ticks([])
        ax.yaxis.set_ticks([])
        for spine in ax.spines.values():
            spine.set_visible(False)
        predicted = class_names[y_pred[i+start_i]]
        match = predicted ==  class_names[y_test[start_i + i]]
        ec = (1, .5, .5)
        fc = (1, .8, .8)
        if match:
            ec = (0, .6, .1)
            fc = (0, .7, .2)
        # predicted label
        ax.text(0, 140, 'P: ' + predicted, size=10, rotation=0,
            ha="left", va="top",
             bbox=dict(boxstyle="round",
                   ec=ec,
                   fc=fc,
                   )
             )
        if not match:
            # true label
            ax.text(0, 170, 'A: ' + class_names[y_test[start_i + i]], size=10, rotation=0,
                ha="left", va="top",
                 bbox=dict(boxstyle="round",
                       ec=ec,
                       fc=fc,
                       )
                 )
    plt.subplots_adjust(left=0, wspace=1, hspace=0)
    plt.show()

In [ ]:
# change the first parameter to the class label you would like to examine.
# if you would like to check different images of the class increase the page number
show_images_prediction('tacos', y_test, page=1)